In [1]:
import numpy as np
from src.vae_model import vae_model
from src.lstm_mapper_model import mapper_model
from src.util import (
    seq_data,
    load_sim_data,  
    load_exp_data,
)
from pyeit import mesh
from src.performance_eval_2D import (compute_evaluation_metrics,
    plot_random_deviations, 
    plot_random_recon_examples, 
    plot_deviations_x_y,
    plot_deviations_perm,
    merge_plots_to_pdf,
    run_deviation_analysis,
    plot_polar_metrics,
    compute_polar_metrics
    )

2025-01-17 09:08:47.801591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737104927.816293 1976251 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737104927.820712 1976251 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 09:08:47.837852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0000 00:00:1737104936.956365 1976251 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libr

In [2]:
n_el = 32
from pyeit import mesh
mesh_obj = mesh.create(n_el, h0=0.05)

## Circle interpolation test: r=0.5 using LSTM trained on r=0.25 & 0.75 (using sim data)

**Loading pretrained models**

In [3]:
lstm_mapper = mapper_model()
#lstm_mapper.summary()
lstm_mapper.load_weights("weights/lstm_circle_interpolation_025_075_med.weights.h5")
print("Gewichte erfolgreich geladen.")

Gewichte erfolgreich geladen.


In [4]:
vae = vae_model()  
vae.build(input_shape=(None, 2840)) 
vae.load_weights("weights/vae2D_best_6000.weights.h5")
print("Gewichte erfolgreich geladen.")

Gewichte erfolgreich geladen.


**Loading test sim data**

In [5]:
voltage_test_sim, gamma_test_sim , _ = load_sim_data("data53")  #todo: adjust test data
#voltage_test_sim, gamma_test_sim , _ = load_sim_data("data48")  #todo: adjust test data
voltage_test_normalized = (voltage_test_sim - np.mean(voltage_test_sim, axis=0)) / np.std(voltage_test_sim, axis=0)

voltage_test = voltage_test_normalized.reshape(-1, 32, 32, 1)  
gamma_test = gamma_test_sim.reshape(-1, 2840)

voltage_test_seq, gamma_true_test = seq_data(voltage_test, gamma_test, n_seg=4)

**Reconstruction using voltage data**

In [6]:
pred_latent = lstm_mapper.predict(voltage_test_seq)
gamma_pred_test = vae.decoder.predict(pred_latent).reshape(-1, 2840)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


## Circle interpolation test: r=0.5 using LSTM trained on r=0.2, 0.3, 0.4, 0.6, 0.7, 0.8 (using sim data)

In [7]:
# Überprüfen Sie zunächst die Shapes ihrer Eingabedaten
print("Input shapes:")
print(f"gamma_true_test shape: {gamma_true_test.shape}")
print(f"gamma_pred_test shape: {gamma_pred_test.shape}")

# Dann berechnen Sie die Metriken
r_true, r_pred, theta_true, theta_pred = compute_polar_metrics(mesh_obj, gamma_true_test, gamma_pred_test)

# Erstellen Sie den Plot
plot_polar_metrics(r_true, r_pred, theta_true, theta_pred)

Input shapes:
gamma_true_test shape: (4996, 2840)
gamma_pred_test shape: (4996, 2840)


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
#plot für Radius usw. wie im Paper